In [7]:
#Jupyter_DASH
from jupyter_dash import JupyterDash
import numpy as np
import pandas as pd
import dash
import dash_core_components as dcc
import plotly.express as px
import plotly.graph_objects as go
import dash_html_components as html
from dash.dependencies import Input, Output
import math

app = JupyterDash(__name__)

#To change colors and font
##colors = {
    #'background': '#111111',
    #'text': '#7FDBFF'
#}

df = pd.read_excel('cancer_mama.xlsx', squeeze=True)

df['age_at_diagnosis'] = np.round(df['age_at_diagnosis'])

df = df.drop(df.columns[range(520, 693)], axis=1)   #se eliminan columnas con muy poca informacion

survivalGraph = go.Bar(
    x=df['age_at_diagnosis'],
    y=df['overall_survival'],
    name='Age vs Survival'
)

chemotherapyGraph = go.Bar(
    x=df['cellularity'],
    y=df['chemotherapy'],
    name='Cellularity vs Chemotherapy'
)

# App layout
app.layout = html.Div([

    html.H1("Factors Involved in Breast Cancer Patient Outcome", style={'text-align': 'center'}),
    html.Br(),

    html.H2('Filter Panel'), 
     html.Br(),
  #Dropdown en value tenemos el valor por default  
    dcc.Dropdown(id="cancer_type",
                 options=[
                     {"label": "Breast Invasive Ductal Carcinoma", "value": "Breast Invasive Ductal Carcinoma"},
                     {"label": "Breast Mixed Ductal and Lobular Carcinoma", "value": "Breast Mixed Ductal and Lobular Carcinoma"},
                     {"label": "Breast Invasive Lobular Carcinoma", "value": "Breast Invasive Lobular Carcinoma"}],
                 multi=False,
                 value="Breast Invasive Ductal Carcinoma",
                 style={'width': "50%"}
                 ),

    html.Div(id='output_container', children=[]),
    html.Br(),
    
    html.H2('Cellularity of Pacients per Chemotherapy'), 
    html.Br(),
    dcc.Graph(id='bar_plot_chemotherapy', figure=go.Figure(data=chemotherapyGraph, layout=go.Layout(barmode='stack'))),

    dcc.Graph(id='scatterplot', figure={}),
    
    html.H2('Overall Survival of Pacients per Age'), 
    html.Br(),
    dcc.Graph(id='bar_plot_survival', figure=go.Figure(data=survivalGraph, layout=go.Layout(barmode='stack')))

])

#Callback
#Si solo hay un output, no se necesita usar paréntesis cuadrados
@app.callback(
    [Output(component_id='output_container', component_property='children'),
     Output(component_id='scatterplot', component_property='figure')],
    [Input(component_id='cancer_type', component_property='value')]
)
#Se crea un def por cada input
def update_graph(option_slctd):
    #Se crea el container, un objeto que forma parte del output y produce la oración de abajo
    container = "The cancer type choseen by the user was: {}".format(option_slctd)

    #Copia del dataframe para no tocar el data frame y jugar con esto
    dff = df.copy()
    dff = dff[dff["cancer_type_detailed"] == option_slctd]
    #A row in which we set that only rows with x column in y value are shown
    #dff = dff[dff["Affected by"] == "Varroa_mites"]

    # Plotly Express para crear la figura > Scatter plot
    fig = px.scatter(dff, x="cellularity", y="chemotherapy", 
                     size='cellularity', color="cancer_type_detailed", hover_name="patient_id", 
                     log_x=True, size_max=55)

    fig.update_layout(transition_duration=500)
    

    # Plotly Graph Objects (GO)
    # fig = go.Figure(
    #     data=[go.Choropleth(
    #         locationmode='USA-states',
    #         locations=dff['state_code'],
    #         z=dff["Pct of Colonies Impacted"].astype(float),
    #         colorscale='Reds',
    #     )]
    # )
    #
    # fig.update_layout(
    #     title_text="Bees Affected by Mites in the USA",
    #     title_xanchor="center",
    #     title_font=dict(size=24),
    #     title_x=0.5,
    #     geo=dict(scope='usa'),
    # )

    #Se debe de retornar la cantidad de outputs que se tengan
    return container, fig


# ------------------------------------------------------------------------------
if __name__ == '__main__':
    app.run_server(debug=True)


Dash app running on http://127.0.0.1:8050/
